In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

from IPython.display import display

from sklearn.svm import OneClassSVM
from numpy import where

dataset = pd.read_csv('emails.csv')
dataset.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [2]:
data = dataset.copy().drop(columns='Email No.')
train, test = train_test_split(data, test_size=0.2, random_state=21)

X_train, y_train = train.drop(columns=['Prediction']) ,train['Prediction']
X_test, y_test = test.drop(columns=['Prediction']), test['Prediction']

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

cls = LogisticRegression(max_iter=4000)
cls.fit(X_train, y_train)
print(classification_report(y_test, cls.predict(X_test)))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       736
           1       0.94      0.96      0.95       299

    accuracy                           0.97      1035
   macro avg       0.96      0.97      0.97      1035
weighted avg       0.97      0.97      0.97      1035



In [5]:
import pickle
# save the model to disk
pickle.dump(cls, open('models/logistic_regression_model.pkl', 'wb'))

In [12]:
def preprocess_input_string(input_string, columns):
    # Tokenize the input string and count the occurrence of each word
    word_list = input_string.lower().split()
    word_count = {word: word_list.count(word) for word in word_list}

    # Create a DataFrame with zeros for all columns
    input_df = pd.DataFrame(columns=columns)
    input_df.loc[0] = [0] * len(columns)

    # Update the DataFrame with word counts from the input string
    for word in word_count:
        if word in input_df.columns:
            input_df.at[0, word] = word_count[word]

    return input_df

def predict_string(input_string):
    # Load the trained model
    loaded_model = pickle.load(open('models/logistic_regression_model.pkl', 'rb'))

    # Preprocess the input string to match the dataset's format
    processed_input_df = preprocess_input_string(input_string, X_train.columns)

    # Make a prediction using predict_proba
    probabilities = loaded_model.predict_proba(processed_input_df)

    # Extract the probability of the positive class
    positive_class_probability = probabilities[0][1]
    return positive_class_probability

In [76]:
# expect gary may's email to have a really low probability of being spam
test_string = "Welcome Aggies! We are excited to welcome both new and continuing students to campus and the community. We wish you a safe and productive school year filled with academic and personal success. UC Davis has many important health and safety resources, so please take time to review this information in case you ever need to access them. Mental Health Resources Aggie Mental Health is a campuswide initiative supporting student mental health and well-being. This campaign is a collaboration between Student Health and Counseling Services’ (SHCS) Health Education and Promotion department and the Aggie Mental Health Ambassador program. Visit the Aggie Mental Health website for help navigating all of campus's many mental health resources, including Counseling Services. You can also follow the Aggie Mental Health Instagram for up to date information about mental health resources on campus. Alcohol, Tobacco and Other Drugs Risk Reduction SHCS’ Health Education and Promotion department offers tips and resources around alcohol and other substances. The Safe Party website and Instagram account (@ucdsafeparty) feature information around: Signs of alcohol poisoning and other overdose resources, including how to obtain Narcan on campus if you are concerned about a friend’s opioid use Navigating consent while drinking and other sexual well-being resources How to be an ‘Upstander’ and/or Effective Wingperson To remove the fear of getting into trouble that may keep students from calling for medical help in an emergency related to alcohol or drugs, UC Davis has the Aggies Act protocol where neither the student who called nor the student who experienced the medical emergency would be subject to the formal student conduct process. A similar law exists in the state of California for students who live off campus. For any students in recovery from alcohol or other substances, UC Davis has a collegiate recovery group, Aggies For Recovery. Find out more on our SHCS recovery resources page. Harassment and Discrimination UC Davis takes all complaints of harassment and discrimination very seriously. The Harassment & Discrimination Assistance and Prevention Program (HDAPP) handles all reports of harassment and discrimination including reports of sexual violence and sexual harassment and can answer any questions you have about how the University responds to such concerns. If you or someone you know experienced harassment or discrimination at UC Davis, please report it here. Sexual Violence Resources If you or someone you know is experiencing sexual violence, including sexual assault, sexual harassment, sexual exploitation, intimate partner violence, stalking or harassment based on gender identity or sexual orientation, there are professional and experienced staff available to support and help you understand your rights and options. All of the confidential resources below are free of cost and available to survivors regardless of their decision to make any formal report. The confidential CARE Advocates provide crisis intervention and advocacy, assist with safety planning and discuss reporting options. They can be reached by calling (530) 752-3299. For non-urgent situations, you can email ucdcare@ucdavis.edu. UC Davis Counseling Services offers confidential short-term counseling to any student experiencing stress or trauma related to a variety of situations. To schedule a counseling appointment, call (530) 752-0871. The Women's Resources and Research Center (WRRC), Lesbian, Gay, Bisexual, Transgender, Queer, Intersex, Asexual Resource Center (LGBTQIA RC) and Ombuds are also designated as confidential resources for survivors of sexual violence. Hazing Resources Stop Hazing is a campuswide campaign developed to provide support and resources for students who are being hazed, empower our community to identify, report and discourage hazing and offer ideas of how to implement positive strategies to bring groups together. The campaign includes the new Stop Hazing website serving as hub for educational resources. Working together, our goal is to create the best environment for our students, faculty, staff and other community residents. We also encourage you to reach out to any of these resources if you ever need support during your time at UC Davis. In health, Cory Vu Associate Vice Chancellor for Health, Wellness and Divisional Resources Division of Student Affairs Raeann Bowlds Director, Health Education and Promotion UC Davis Student Health and Counseling Services "
probability = predict_string(test_string)
print("Probability of being positive spam:", probability)

Probability of being positive spam: 2.0980016736801516e-12
Probability of being positive spam: 2.877286656230711e-08
